<h1>语言模型测试<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#使用BERT-seq" data-toc-modified-id="使用BERT-seq-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>使用BERT-seq</a></span><ul class="toc-item"><li><span><a href="#正常训练" data-toc-modified-id="正常训练-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>正常训练</a></span></li><li><span><a href="#冻结模型" data-toc-modified-id="冻结模型-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>冻结模型</a></span><ul class="toc-item"><li><span><a href="#封装" data-toc-modified-id="封装-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>封装</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>训练</a></span></li></ul></li><li><span><a href="#半精度float16(无冻结)" data-toc-modified-id="半精度float16(无冻结)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>半精度float16(无冻结)</a></span></li><li><span><a href="#float16+冻结模型" data-toc-modified-id="float16+冻结模型-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>float16+冻结模型</a></span></li><li><span><a href="#分布式训练" data-toc-modified-id="分布式训练-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>分布式训练</a></span></li><li><span><a href="#GPU使用情况" data-toc-modified-id="GPU使用情况-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>GPU使用情况</a></span><ul class="toc-item"><li><span><a href="#释放显存" data-toc-modified-id="释放显存-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>释放显存</a></span></li></ul></li></ul></li><li><span><a href="#使用bert-Model" data-toc-modified-id="使用bert-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>使用bert Model</a></span></li><li><span><a href="#自定义self-attention-encoder" data-toc-modified-id="自定义self-attention-encoder-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>自定义self-attention encoder</a></span></li><li><span><a href="#自定义bert-model" data-toc-modified-id="自定义bert-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>自定义bert-model</a></span></li><li><span><a href="#试验:-tokenizer和BERT-NER" data-toc-modified-id="试验:-tokenizer和BERT-NER-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>试验: tokenizer和BERT-NER</a></span></li><li><span><a href="#试验:-tokenizer和Spacy-NER" data-toc-modified-id="试验:-tokenizer和Spacy-NER-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>试验: tokenizer和Spacy-NER</a></span></li><li><span><a href="#XLNET测试" data-toc-modified-id="XLNET测试-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>XLNET测试</a></span></li><li><span><a href="#试验:-BertForQuestionAnswering" data-toc-modified-id="试验:-BertForQuestionAnswering-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>试验: BertForQuestionAnswering</a></span><ul class="toc-item"><li><span><a href="#配置小型模型" data-toc-modified-id="配置小型模型-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>配置小型模型</a></span></li></ul></li><li><span><a href="#测试Dataset与字符串" data-toc-modified-id="测试Dataset与字符串-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>测试Dataset与字符串</a></span></li><li><span><a href="#nomalizations比较" data-toc-modified-id="nomalizations比较-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>nomalizations比较</a></span></li><li><span><a href="#Albert" data-toc-modified-id="Albert-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Albert</a></span></li><li><span><a href="#RoBerta" data-toc-modified-id="RoBerta-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>RoBerta</a></span></li></ul></div>

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/folders/')
    # !pip install transformers
    # !pip install -U spacy[cuda100]
    # !wget -P /content/folders/My\ Drive/download/ https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz
    # !pip install /content/folders/My\ Drive/download/en_core_web_lg-2.2.5.tar.gz
    # !wget -P /content/folders/My\ Drive/HotpotQA/ http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
    # json_train_path = '/content/folders/My Drive/HotpotQA/样例_hotpot_train_v1.1.json' # 5个例子
    json_train_path = '/content/folders/My Drive/HotpotQA/hotpot_train_v1.1.json'
    save_cache_path = '/content/folders/My Drive/save_cache/'
    save_cache_path_linux = '/content/folders/My\ Drive/save_cache/'
    HotpotQA_path = '/content/folders/My Drive/HotpotQA'
    proxies = None
except:
    json_train_path = r'./data/hotpot_train_v1.1.json'
    HotpotQA_path = './'
    save_cache_path = 'save_cache/'
    use_proxy = False
    proxies={"http_proxy": "127.0.0.1:10802",
        "https_proxy": "127.0.0.1:10802"} if use_proxy else None


# 使用BERT-seq

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [2]:
from transformers import BertForSequenceClassification
bert_BASE_Seq = BertForSequenceClassification.from_pretrained('bert-base-uncased')
_ = bert_BASE_Seq.train()
_ = bert_BASE_Seq.to('cuda')

In [3]:
class Test_Dateset(Dataset):
    def __init__(self, max_num=5000, max_seq=512, vocab_size=30522, label_num=2):
        self.max_num = max_num
        self.max_seq = max_seq
        self.vocab_size = vocab_size
        self.label_num = label_num
        
    def __len__(self):
        return self.max_num
    
    def __getitem__(self, index):
        data = torch.randint(0,self.vocab_size,(self.max_seq,), device='cuda').unsqueeze(0)
        labels = torch.randint(0,self.label_num-1,(1,), device='cuda')
        return {'input_ids':data,
                'labels':labels}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cuda'): 
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, _tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].squeeze(1).to(device)
        yield out_data_dict

def compute_accuracy(logits, y_target):
    _, logits_indices = logits.max(dim=1)
    n_correct = torch.eq(logits_indices, y_target).sum().item()
    return n_correct / len(logits_indices) * 100

In [4]:
dateset_01 = Test_Dateset(max_seq=200)

test_logits = torch.randn([5,2])
test_labels = torch.randint(0,1,[5,1])
loss_fn = nn.CrossEntropyLoss()

In [5]:
test_labels.squeeze_(-1).shape

torch.Size([5])

In [6]:
loss_fn(test_logits,test_labels)

tensor(0.7182)

In [7]:
bert_BASE_Seq(**dateset_01.__getitem__(1))

(tensor(0.7058, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor([[0.2318, 0.2569]], device='cuda:0', grad_fn=<AddmmBackward>))

## 正常训练

In [9]:
batch_size = 26 #  10311MiB / 11019MiB
epoch_bar = tqdm_notebook(desc='training routine',
                total=dateset_01.get_num_batches(batch_size),
                position=0)
optimizer = optim.Adam(bert_BASE_Seq.parameters(), 0.001)

for i,batch_dict in enumerate(generate_batches(dateset_01,batch_size)):
    optimizer.zero_grad()

    y_target = batch_dict.pop('labels')
    res_tuple = bert_BASE_Seq(**batch_dict)
    logits = res_tuple[0]

    loss = nn.CrossEntropyLoss()(logits, y_target)

    loss.backward()

    optimizer.step()

    acc = compute_accuracy(logits, y_target)

    epoch_bar.set_postfix(loss=loss.item(), acc=acc, batch=i)
    epoch_bar.update()

# 192/192 [01:13<00:00, 2.60it/s, acc=100, batch=191, loss=1e-5]

In [0]:
for i in bert_BASE_Seq.parameters():
    print(i.data.shape)
    print(i.requires_grad)
    break

## 冻结模型

以`bert-base-uncased`为例.

- `model.parameters()`
    - 返回tensor的子类, 有data和requires_grad.
    - 直接设置grad可以不用返回model.


- `model.state_dict()`
    - 返回有序字典
    - key是该层的name.
    

- `model.children()`
    - 树状结构. root下有3个children.
    1. 11个attention层.
    2. Dropout(p=0.1, inplace=False)
    3. Linear(in_features=768, out_features=2, bias=True)


In [9]:
for index, layer in enumerate(bert_BASE_Seq.parameters()):
    print(layer)
    break

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       device='cuda:0', requires_grad=True)


In [10]:
# 返回orderDict. k,v分别是名称和tensor.
FLAG_no_grad = True
for index, (key, value) in enumerate(bert_BASE_Seq.state_dict().items()):
    if FLAG_no_grad: value.requires_grad_ = True
    if key.startswith('bert.encoder.layer.10'): FLAG_no_grad = False

In [11]:
for (k, v) in bert_BASE_Seq.state_dict().items():
    print(k ,v.requires_grad)
    break

bert.embeddings.word_embeddings.weight False


In [12]:
# torch.nn.parameter.Parameter  are Tensor subclasses
# data and requires_grad 
# 方法可行
for index, i in enumerate(bert_BASE_Seq.parameters()):
    if index > 1 :
        i.requires_grad = True
    else:
        i.requires_grad = False
    
    if index > 4: break
    

for index, i in enumerate(bert_BASE_Seq.parameters()):
    if index > 3: break
    print(index, i.requires_grad)


0 False
1 False
2 True
3 True


### 封装

In [13]:
def freeze_to_layer(model, layer_name):
    '''冻结层. 从0到layer_name.'''
    
    if layer_name == 'all':
        index_start = len(model.state_dict())
    else:
        index_start = -1
        for index, (key, _value) in enumerate(model.state_dict().items()):
            if key.startswith(layer_name): 
                index_start = index
                break

    if index_start < 0:
        print(f"Don't find layer name: {layer_name}")
        return
    
    no_grad_nums = index_start + 1
    grad_nums = 0

    for index, i in enumerate(model.parameters()):
        if index >= index_start:
            i.requires_grad = True
            grad_nums += 1
        else:
            i.requires_grad = False
    
    print(f"freeze layers num: {no_grad_nums}, active layers num: {grad_nums}.")
    # no need to return.

In [14]:
freeze_to_layer(bert_BASE_Seq,'bert.encoder.layer.10')

freeze layers num: 166, active layers num: 36.


### 训练

In [15]:
batch_size = 128 # 几乎极限 10741MiB / 11019MiB
epoch_bar = tqdm_notebook(desc='training routine',
                total=dateset_01.get_num_batches(batch_size),
                position=0)
optimizer = optim.Adam(filter(lambda p: p.requires_grad, bert_BASE_Seq.parameters()), 0.001)

for i,batch_dict in enumerate(generate_batches(dateset_01,batch_size)):
    optimizer.zero_grad()

    y_target = batch_dict.pop('labels')
    res_tuple = bert_BASE_Seq(**batch_dict)
    logits = res_tuple[0]

    loss = nn.CrossEntropyLoss()(logits, y_target)

    loss.backward()

    optimizer.step()

    acc = compute_accuracy(logits, y_target)

    epoch_bar.set_postfix(loss=loss.item(), acc=acc, batch=i)
    epoch_bar.update()

# 39/39 [00:29<00:00, 1.32it/s, acc=100, batch=38, loss=2.76e-7]

## 半精度float16(无冻结)

In [8]:
from apex import amp

In [9]:
batch_size = 32 # 9527MiB / 11019MiB
epoch_bar = tqdm_notebook(desc='training routine',
                total=dateset_01.get_num_batches(batch_size),
                position=0)
optimizer = optim.Adam(bert_BASE_Seq.parameters(), 0.001)

# add.
bert_BASE_Seq, optimizer = amp.initialize(bert_BASE_Seq, optimizer)

for i,batch_dict in enumerate(generate_batches(dateset_01,batch_size)):
    optimizer.zero_grad()

    y_target = batch_dict.pop('labels')
    res_tuple = bert_BASE_Seq(**batch_dict)
    logits = res_tuple[0]

    loss = nn.CrossEntropyLoss()(logits, y_target)

    # add.
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    
    optimizer.step()

    acc = compute_accuracy(logits, y_target)

    epoch_bar.set_postfix(loss=loss.item(), acc=acc, batch=i)
    epoch_bar.update()
    
# 156/156 [00:40<00:00, 3.84it/s, acc=100, batch=155, loss=0]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


## float16+冻结模型

In [8]:
def freeze_to_layer(model, layer_name):
    '''冻结层. 从0到layer_name.'''
    index_start = -1
    for index, (key, _value) in enumerate(model.state_dict().items()):
        if key.startswith(layer_name): 
            index_start = index
            break
    
    if index_start < 0:
        print(f"Don't find layer name: {layer_name}")
        return
    
    no_grad_nums = index_start + 1
    grad_nums = 0

    for index, i in enumerate(model.parameters()):
        if index >= index_start:
            i.requires_grad = True
            grad_nums += 1
        else:
            i.requires_grad = False
    
    print(f"freeze layers num: {no_grad_nums}, active layers num: {grad_nums}.")
    # no need to return.
freeze_to_layer(bert_BASE_Seq,'bert.encoder.layer.10')

freeze layers num: 166, active layers num: 36.


In [9]:
from apex import amp

In [10]:
batch_size = 212 # 10931MiB / 11019MiB
epoch_bar = tqdm_notebook(desc='training routine',
                total=dateset_01.get_num_batches(batch_size),
                position=0)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, bert_BASE_Seq.parameters()), 0.001)

# add.
bert_BASE_Seq, optimizer = amp.initialize(bert_BASE_Seq, optimizer)

for i,batch_dict in enumerate(generate_batches(dateset_01,batch_size)):
    optimizer.zero_grad()

    y_target = batch_dict.pop('labels')
    res_tuple = bert_BASE_Seq(**batch_dict)
    logits = res_tuple[0]

    loss = nn.CrossEntropyLoss()(logits, y_target)

    # add.
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    
    optimizer.step()

    acc = compute_accuracy(logits, y_target)

    epoch_bar.set_postfix(loss=loss.item(), acc=acc, batch=i)
    epoch_bar.update()
    
# 23/23 [00:13<00:00, 1.66it/s, acc=100, batch=22, loss=4.5e-9]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


## 分布式训练

需要使用命令行.

## GPU使用情况

In [12]:
def print_gpu_info(_print = True):
    if not _print: return None
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
        print('and then re-execute this cell.')
    else:
        print(gpu_info)

# 参数量
import numpy as np
def model_parameters(model, type_size=4):
    para = sum([np.prod(list(p.size())) for p in model.parameters()])
    print('Model {} : params: {:4f}M'.format(model._get_name(), para * type_size / 1000 / 1000))

In [13]:
model_parameters(bert_BASE_Seq)

Model BertForSequenceClassification : params: 437.935112M


In [14]:
print_gpu_info()

Sun Mar  8 00:51:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 440.36       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:19:00.0 Off |                  N/A |
| 45%   47C    P2    45W / 250W |  10075MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   33C    P8     4W / 250W |     12MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

### 释放显存

In [15]:
torch.cuda.empty_cache()
print_gpu_info()

Sun Mar  8 00:51:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 440.36       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:19:00.0 Off |                  N/A |
| 45%   47C    P2    53W / 250W |   2679MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   33C    P8     5W / 250W |     12MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

# 使用bert Model

In [7]:
from transformers import AutoConfig, AutoModel

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [11]:
model_name = 'bert-base-uncased'
my_config = AutoConfig.from_pretrained(model_name)
model_bert = AutoModel.from_config(my_config)

In [18]:
_ = model_bert.to('cuda')

In [14]:
class Test_Dateset(Dataset):
    def __init__(self, max_num=5000, max_seq=512, vocab_size=30522):
        self.max_num = max_num
        self.max_seq = max_seq
        self.vocab_size = vocab_size
        
    def __len__(self):
        return self.max_num
    
    def __getitem__(self, index):
        data = torch.randint(0,self.vocab_size,(self.max_seq,), device='cuda').unsqueeze(0)
        return {'input_ids':data}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

In [15]:
dateset_02 = Test_Dateset(max_seq=200)

In [20]:
last_hidden_state , pooler_output  = model_bert(**dateset_02.__getitem__(1))

In [21]:
last_hidden_state.shape

torch.Size([1, 200, 768])

In [23]:
pooler_output.shape

torch.Size([1, 768])

# 自定义self-attention encoder


In [ ]:
# 给定变长features 返回padding和mask

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time

In [39]:
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        # 第二个参数是匿名函数 -- 因为sublayer第二个参数直接拿来调用.
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)
    
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # mask == 0是boolen矩阵, 标明哪些位置是0. 这里就是直接对mask为true的部分取-1e9.
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        # -1应该是句子长度.
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)
    
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))
    
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)
    
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # 不再求导.
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
    
def make_model(N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N)

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [61]:
tmp_model = make_model(3, d_model=768)

In [64]:
# 封装
def build_padding(batch_seq_dim, seq_max = 19):
    x_padding = []
    x_mask = []
    for t in batch_seq_dim:
        tokens_len = t.shape[-2]
        x_mask.append([1]*tokens_len + [0]*(seq_max - tokens_len))
        pd = (0,0,0, seq_max - tokens_len)
        x_padding.append(F.pad(t, pd ,"constant", 0))

    x_padding = torch.cat(x_padding, dim=0)
    x_mask = torch.tensor(x_mask).unsqueeze(-2)
    
    return x_padding, x_mask

In [78]:
seq_max = 512
batch_seq_dim = [torch.randn((1,np.random.randint(0,seq_max),768)) for i in range(10)]
x_test, x_mask = build_padding(batch_seq_dim, seq_max)

In [79]:
x_res = tmp_model(x_test, x_mask)

In [80]:
x_res.shape

torch.Size([10, 512, 768])

# 自定义bert-model

使用`config`自定义参数, 不使用预训练模型. 传入的是mask的`[bsz, seq, dim]`

In [46]:
from transformers import AutoConfig, AutoModel, BertConfig

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

In [70]:
my_config = BertConfig(max_length=1024,
                      num_hidden_layers = 3,
                      num_attention_heads = 8,
                      max_position_embeddings = 1024)


In [71]:
model_bert = AutoModel.from_config(my_config)

In [81]:
model_inputs = {'inputs_embeds':x_test,
               'attention_mask':x_mask}

In [73]:
last_hidden_state, pooler_output = model_bert(**model_inputs)

In [74]:
last_hidden_state.shape

torch.Size([10, 1024, 768])

In [75]:
pooler_output.shape

torch.Size([10, 768])

# 试验: tokenizer和BERT-NER

使用XLNET做分词,使用BERT-NER进行命名实体识别. **不行!!**. BERT-NER本身最长输入就是512.

In [90]:
model_name = 'xlnet-base-cased'

In [91]:
from transformers import AutoConfig, AutoModel, BertConfig

In [92]:
proxies={"http_proxy": "127.0.0.1:10802",
"https_proxy": "127.0.0.1:10802"}

config = AutoConfig.from_pretrained(model_name, proxies=proxies)

In [114]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_NER = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english",
                                                           proxies = proxies)

tokenizer_XLNET = AutoTokenizer.from_pretrained(model_name,proxies = proxies)

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased-whole-word-masking",
                                         proxies = proxies)

In [115]:
max_sen_in_train = """The family first appeared in 2006 and the family have been involved in a number of the show's most high-profile storylines, most notably John Paul McQueen's (James Sutton) affair with Craig Dean (Guy Burnet); Jacqui McQueen's (Claire Cooper) whirlwind relationship with Tony Hutchinson (Nick Pickard); Myra McQueen's (Nicole Barber-Lane) long-lost son Niall Rafferty's (Barry Sloane) revenge on his family by holding them hostage in an abandoned church and blowing it up, ultimately killing his half-sister Tina Reilly (Leah Hackett); Theresa McQueen's (Jorgie Porter) pregnancy by her cousin Carmel McQueen's (Gemma Merna) fiancé Calvin Valentine (Ricky Whittle) and later shooting him dead on their wedding day; Mercedes McQueen's (Jennifer Metcalfe) affair with her fiancé Riley Costello's (Rob Norbuy) father Carl (Paul Opacic); being kidnapped by Riley's grandfather Silas; staking Riley's second cousin Mitzee Minniver; Jacqui coping with the death of her husband Rhys Ashworth (Andrew Moss) in a bus crash, learning that he had been having an affair with Cindy Cunningham (Stephanie Waring) and that he got Sinead O'Connor (Stephanie Davis) pregnant; Mercedes stalking Mitzeee (Rachel Shenton) and stabbing herself and framing her; Carmel's facial disfigurement; Myra faking her own death to escape her daughter Mercedes' evil husband, Dr. Paul Browning (Joseph Thompson); Mercedes killing her husband Doctor Browning by striking him over the head with a shovel; John Paul's male rape at the hands of his pupil Finn O'Connor (Keith Rice); the train crash which ultimately killed Carmel; Mercedes faking her death to help Grace Black (Tamara Wall) get revenge on Freddie Roscoe (Charlie Clapham); Theresa donating her kidney to Nico Blake (Persephone Swales-Dawson); Porsche (Twinnie Lee Moore) and Cleo McQueen's (Nadine Rose Mulkerrin) sexual abuse at the hands of their mother Reenie McQueen's (Zöe Lucker) fiancé Pete Buchanan (Kai Owen); Phoebe McQueen's (Mandip Gill) murder in hospital by the Gloved Hand Killer; the stillbirth of Mercedes' baby Gabriel McQueen; John Paul's transgender boss Sally St. Claire (Annie Wallace) being revealed as his biological father, Mercedes being framed for drugs by Joanne Cardsley (Rachel Leskovac), Celine McQueen (Sarah George) and Diego Salvador Martinez Hernandez De La Cruz (Juan Pablo Yepez)'s sham wedding for money and Celine being murdered by her ex-boyfriend and serial killer Cameron Campbell (Cameron Moore) after discover he causes the fire at the fair on Halloween 2016."""
max_p = 'Loan modification is the systematic alteration of mortgage loan agreements that help those having problems making the payments by reducing interest rates, monthly payments or principal balances. Lending institutions could make one or more of these changes to relieve financial pressure on borrowers to prevent the condition of foreclosure. Loan modifications have been practiced in the United States since The 2008 Crash Of The Housing Market from Washington Mutual, Chase Home Finance, Chase, JP Morgan & Chase, other contributors like MER\'s. Crimes of Mortgage ad Real Estate Staff had long assisted nd finally the squeaky will could not continue as their deviant practices broke the state and crashed. Modification owners either ordered by The United States Department of Housing, The United States IRS or President Obamas letters from Note Holders came to those various departments asking for the Democratic process to help them keep their homes and protection them from explosion. Thus the birth of Modifications. It is yet to date for clarity how theses enforcements came into existence and except b whom, but t is certain that note holders form the Midwest reached out in the Democratic Process for assistance. FBI Mortgage Fraud Department came into existence. Modifications HMAP HARP were also birthed to help note holders get Justice through reduced mortgage by making terms legal. Modification of mortgage terms was introduced by IRS staff addressing the crisis called the HAMP TEAMS that went across the United States desiring the new products to assist homeowners that were victims of predatory lending practices, unethical staff, brokers, attorneys and lenders that contributed to the crash. Modification were a fix to the crash as litigation has ensued as the lenders reorganized and renamed the lending institutions and government agencies are to closely monitor them. Prior to modifications loan holders that experiences crisis would use Loan assumptions and Loan transfers to keep the note in the 1930s. During the Great Depression, loan transfers, loan assumption, and loan bail out programs took place at the state level in an effort to reduce levels of loan foreclosures while the Federal Bureau of Investigation, Federal Trade Commission, Comptroller, the United States Government and State Government responded to lending institution violations of law in these arenas by setting public court records that are legal precedence of such illegal actions. The legal precedents and reporting agencies were created to address the violations of laws to consumers while the Modifications were created to assist the consumers that are victims of predatory lending practices. During the so-called "Great Recession" of the early 21st century, loan modification became a matter of national policy, with various actions taken to alter mortgage loan terms to prevent further economic destabilization. Due to absorbent personal profits nothing has been done to educate Homeowners or Creditors that this money from equity, escrow is truly theirs the Loan Note Holder and it is their monetary rights as the real prize and reason for the Housing Crash was the profit n obtaining the mortgage holders Escrow. The Escrow and Equity that is accursed form the Note Holders payments various staff through the United States claimed as recorded and cashed by all staff in real-estate from local residential Tax Assessing Staff, Real Estate Staff, Ordinance Staff, Police Staff, Brokers, attorneys, lending institutional staff but typically Attorneys who are also typically the owners or Rental properties that are trained through Bankruptcies\'. that collect the Escrow that is rightfully the Homeowners but because most Homeowners are unaware of what money is due them and how they can loose their escrow. Most Creditors are unaware that as the note holder that the Note Holder are due a annual or semi annual equity check and again bank or other lending and or legal intuitions staff claim this monies instead. This money Note Holders were unaware of is the prize of real estate and the cause of the Real Estate Crash of 2008 where Lending Institutions provided mortgages to people years prior they know they would eventually loose with Loan holders purchasing Balloon Mortgages lending product that is designed to make fast money off the note holder whom is always typically unaware of their escrow, equity and that are further victimized by conferences and books on HOW TO MAKE MONEY IN REAL STATE - when in fact the money is the Note Holder. The key of the crash was not the House, but the loan product used and the interest and money that was accrued form the note holders that staff too immorally. The immoral and illegal actions of predatory lending station and their staff began with the inception of balloon mortgages although illegal activity has always existed in the arena, yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff, legal staff and others created fraudulent documents to change payments and billing of note holders to obtain the money note holders are typically unaware of) and other watch dog agencies came into existence to examine if houses were purchased through a processed check at Government Debited office as many obtained free homes illegally. Many were incarcerated for such illegal actions. Modifications fixed the Notes to proper lower interest, escrow, tax fees that staff typically raised for no reason. Many people from various arenas involved in reals estate have been incarcerated for these actions as well as other illegal actions like charging for a modification. Additionally Modifications were also made to address the falsifications such as inappropriate mortgage charges, filing of fraudulently deeds, reporting of and at times filing of fraudulent mortgages that were already paid off that were fraudulently continued by lenders staff and attorneys or brokers or anyone in the Real Estate Chain through the issues of real estate terms to continue to violate United States Laws, contract law and legal precedence where collusion was often done again to defraud and steal from the Note Holder was such a common practice that was evidence as to why the Mortgage Crash in 2008 occurred for the purpose of wining the prize of stealing form Homeowners and those that foreclosed was actually often purposefully for these monies note holders were unaware of to be obtained which was why Balloon mortgages and loans were given to the staff in the Real Estate Market with the hoper and the expectation that the loan holders would default as it offered opportunity to commit illegal transactions of obtaining the homeowners funds. While such scams were addressed through modifications in 2008. The Market relied heavily on Consumers ignorance to prosper, ignorance of real estate terms, ignorance on what they were to be charged properly for unethical financial gain and while staff in real estates lending arenas mingled terms to deceive y deliberate confusion consumers out of cash and homes while the USA Government provided Justice through President Obamas Inception and IRS Inception of Modifications which addressed these unethical profits in Reals Estate. It was in 2009 that HARP, HAMP and Modifications were introduced to stop the victimization of Note Holders. Taking on the Banks that ran USA Government was a great and dangerous undertaking that made America Great Again as Justice for Consumers reigned. Legal action taken against institutions that have such business practices can be viewed in State Code of Law and Federal Law on precedent cases that are available to the public. Finally, It had been unlawful to be charged by an attorney to modify as well as fro banking staff to modify terms to increase a mortgage and or change lending product to a balloon in an concerted effort to make homeowner foreclose which is also illegal, computer fraud and not the governments intended purpose or definition of a modification.'
DEVICE = 'cpu'

In [116]:
tokens = tokenizer_XLNET.tokenize(max_sen_in_train)

In [117]:
tokens = ' '.join([i.replace( '▁', '') for i in tokens])

In [118]:
def find_NER_in_Model(content_raw, tokens=None, model=model_NER, tokenizer = tokenizer):
    '''返回: tokens[1:-1], inputs, entities_list
    第一个是不包含[CLS] [SEP]的分词序列(content_tokens是包含的).
    第二个是实体列表.'''
    label_list = [
        "O",       # Outside of a named entity
        "B-MISC",  # Beginning of a miscellaneous entity right after another miscellaneous entity
        "I-MISC",  # Miscellaneous entity
        "B-PER",   # Beginning of a person's name right after another person's name
        "I-PER",   # Person's name
        "B-ORG",   # Beginning of an organisation right after another organisation
        "I-ORG",   # Organisation
        "B-LOC",   # Beginning of a location right after another location
        "I-LOC"    # Location
        ]
    # Bit of a hack to get the tokens with the special tokens
    if not tokens: tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(content_raw)))
    inputs = tokenizer.encode(content_raw, return_tensors='pt').to(DEVICE)
    outputs = model_NER(inputs)[0]
    predictions = torch.argmax(outputs, dim=2)
    # 去除 [cls] [sep]
    res = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())][1:-1]
    # print(predictions)
    # print(res)

    entities_list = []
    cursor_1 = 0
    cursor_2 = 0

    while cursor_1 < len(res):
        entities_dict = {}
        temp = []
        if res[cursor_1][1] == 'O': 
            cursor_1+=1
            continue
        
        entities_dict['type'] = res[cursor_1][1]
        entities_dict['span_start'] = cursor_1
        temp.append(res[cursor_1][0])
        cursor_2 = cursor_1 + 1
        while cursor_2 < len(res):
            if res[cursor_2][1] == 'O':
                cursor_1 = cursor_2 + 1
                break
            
            temp.append(res[cursor_2][0])
            cursor_2 += 1
        
        cursor_1 += cursor_2
        entities_dict['content'] = ' '.join(temp).replace(' ##', '')
        entities_dict['content_tokens'] = temp
        entities_list.append(entities_dict)

    return tokens[1:-1], entities_list

# 试验: tokenizer和Spacy-NER


In [114]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

tokenizer_XLNET = AutoTokenizer.from_pretrained(model_name,proxies = proxies)

In [115]:
max_sen_in_train = """The family first appeared in 2006 and the family have been involved in a number of the show's most high-profile storylines, most notably John Paul McQueen's (James Sutton) affair with Craig Dean (Guy Burnet); Jacqui McQueen's (Claire Cooper) whirlwind relationship with Tony Hutchinson (Nick Pickard); Myra McQueen's (Nicole Barber-Lane) long-lost son Niall Rafferty's (Barry Sloane) revenge on his family by holding them hostage in an abandoned church and blowing it up, ultimately killing his half-sister Tina Reilly (Leah Hackett); Theresa McQueen's (Jorgie Porter) pregnancy by her cousin Carmel McQueen's (Gemma Merna) fiancé Calvin Valentine (Ricky Whittle) and later shooting him dead on their wedding day; Mercedes McQueen's (Jennifer Metcalfe) affair with her fiancé Riley Costello's (Rob Norbuy) father Carl (Paul Opacic); being kidnapped by Riley's grandfather Silas; staking Riley's second cousin Mitzee Minniver; Jacqui coping with the death of her husband Rhys Ashworth (Andrew Moss) in a bus crash, learning that he had been having an affair with Cindy Cunningham (Stephanie Waring) and that he got Sinead O'Connor (Stephanie Davis) pregnant; Mercedes stalking Mitzeee (Rachel Shenton) and stabbing herself and framing her; Carmel's facial disfigurement; Myra faking her own death to escape her daughter Mercedes' evil husband, Dr. Paul Browning (Joseph Thompson); Mercedes killing her husband Doctor Browning by striking him over the head with a shovel; John Paul's male rape at the hands of his pupil Finn O'Connor (Keith Rice); the train crash which ultimately killed Carmel; Mercedes faking her death to help Grace Black (Tamara Wall) get revenge on Freddie Roscoe (Charlie Clapham); Theresa donating her kidney to Nico Blake (Persephone Swales-Dawson); Porsche (Twinnie Lee Moore) and Cleo McQueen's (Nadine Rose Mulkerrin) sexual abuse at the hands of their mother Reenie McQueen's (Zöe Lucker) fiancé Pete Buchanan (Kai Owen); Phoebe McQueen's (Mandip Gill) murder in hospital by the Gloved Hand Killer; the stillbirth of Mercedes' baby Gabriel McQueen; John Paul's transgender boss Sally St. Claire (Annie Wallace) being revealed as his biological father, Mercedes being framed for drugs by Joanne Cardsley (Rachel Leskovac), Celine McQueen (Sarah George) and Diego Salvador Martinez Hernandez De La Cruz (Juan Pablo Yepez)'s sham wedding for money and Celine being murdered by her ex-boyfriend and serial killer Cameron Campbell (Cameron Moore) after discover he causes the fire at the fair on Halloween 2016."""
max_p = 'Loan modification is the systematic alteration of mortgage loan agreements that help those having problems making the payments by reducing interest rates, monthly payments or principal balances. Lending institutions could make one or more of these changes to relieve financial pressure on borrowers to prevent the condition of foreclosure. Loan modifications have been practiced in the United States since The 2008 Crash Of The Housing Market from Washington Mutual, Chase Home Finance, Chase, JP Morgan & Chase, other contributors like MER\'s. Crimes of Mortgage ad Real Estate Staff had long assisted nd finally the squeaky will could not continue as their deviant practices broke the state and crashed. Modification owners either ordered by The United States Department of Housing, The United States IRS or President Obamas letters from Note Holders came to those various departments asking for the Democratic process to help them keep their homes and protection them from explosion. Thus the birth of Modifications. It is yet to date for clarity how theses enforcements came into existence and except b whom, but t is certain that note holders form the Midwest reached out in the Democratic Process for assistance. FBI Mortgage Fraud Department came into existence. Modifications HMAP HARP were also birthed to help note holders get Justice through reduced mortgage by making terms legal. Modification of mortgage terms was introduced by IRS staff addressing the crisis called the HAMP TEAMS that went across the United States desiring the new products to assist homeowners that were victims of predatory lending practices, unethical staff, brokers, attorneys and lenders that contributed to the crash. Modification were a fix to the crash as litigation has ensued as the lenders reorganized and renamed the lending institutions and government agencies are to closely monitor them. Prior to modifications loan holders that experiences crisis would use Loan assumptions and Loan transfers to keep the note in the 1930s. During the Great Depression, loan transfers, loan assumption, and loan bail out programs took place at the state level in an effort to reduce levels of loan foreclosures while the Federal Bureau of Investigation, Federal Trade Commission, Comptroller, the United States Government and State Government responded to lending institution violations of law in these arenas by setting public court records that are legal precedence of such illegal actions. The legal precedents and reporting agencies were created to address the violations of laws to consumers while the Modifications were created to assist the consumers that are victims of predatory lending practices. During the so-called "Great Recession" of the early 21st century, loan modification became a matter of national policy, with various actions taken to alter mortgage loan terms to prevent further economic destabilization. Due to absorbent personal profits nothing has been done to educate Homeowners or Creditors that this money from equity, escrow is truly theirs the Loan Note Holder and it is their monetary rights as the real prize and reason for the Housing Crash was the profit n obtaining the mortgage holders Escrow. The Escrow and Equity that is accursed form the Note Holders payments various staff through the United States claimed as recorded and cashed by all staff in real-estate from local residential Tax Assessing Staff, Real Estate Staff, Ordinance Staff, Police Staff, Brokers, attorneys, lending institutional staff but typically Attorneys who are also typically the owners or Rental properties that are trained through Bankruptcies\'. that collect the Escrow that is rightfully the Homeowners but because most Homeowners are unaware of what money is due them and how they can loose their escrow. Most Creditors are unaware that as the note holder that the Note Holder are due a annual or semi annual equity check and again bank or other lending and or legal intuitions staff claim this monies instead. This money Note Holders were unaware of is the prize of real estate and the cause of the Real Estate Crash of 2008 where Lending Institutions provided mortgages to people years prior they know they would eventually loose with Loan holders purchasing Balloon Mortgages lending product that is designed to make fast money off the note holder whom is always typically unaware of their escrow, equity and that are further victimized by conferences and books on HOW TO MAKE MONEY IN REAL STATE - when in fact the money is the Note Holder. The key of the crash was not the House, but the loan product used and the interest and money that was accrued form the note holders that staff too immorally. The immoral and illegal actions of predatory lending station and their staff began with the inception of balloon mortgages although illegal activity has always existed in the arena, yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff, legal staff and others created fraudulent documents to change payments and billing of note holders to obtain the money note holders are typically unaware of) and other watch dog agencies came into existence to examine if houses were purchased through a processed check at Government Debited office as many obtained free homes illegally. Many were incarcerated for such illegal actions. Modifications fixed the Notes to proper lower interest, escrow, tax fees that staff typically raised for no reason. Many people from various arenas involved in reals estate have been incarcerated for these actions as well as other illegal actions like charging for a modification. Additionally Modifications were also made to address the falsifications such as inappropriate mortgage charges, filing of fraudulently deeds, reporting of and at times filing of fraudulent mortgages that were already paid off that were fraudulently continued by lenders staff and attorneys or brokers or anyone in the Real Estate Chain through the issues of real estate terms to continue to violate United States Laws, contract law and legal precedence where collusion was often done again to defraud and steal from the Note Holder was such a common practice that was evidence as to why the Mortgage Crash in 2008 occurred for the purpose of wining the prize of stealing form Homeowners and those that foreclosed was actually often purposefully for these monies note holders were unaware of to be obtained which was why Balloon mortgages and loans were given to the staff in the Real Estate Market with the hoper and the expectation that the loan holders would default as it offered opportunity to commit illegal transactions of obtaining the homeowners funds. While such scams were addressed through modifications in 2008. The Market relied heavily on Consumers ignorance to prosper, ignorance of real estate terms, ignorance on what they were to be charged properly for unethical financial gain and while staff in real estates lending arenas mingled terms to deceive y deliberate confusion consumers out of cash and homes while the USA Government provided Justice through President Obamas Inception and IRS Inception of Modifications which addressed these unethical profits in Reals Estate. It was in 2009 that HARP, HAMP and Modifications were introduced to stop the victimization of Note Holders. Taking on the Banks that ran USA Government was a great and dangerous undertaking that made America Great Again as Justice for Consumers reigned. Legal action taken against institutions that have such business practices can be viewed in State Code of Law and Federal Law on precedent cases that are available to the public. Finally, It had been unlawful to be charged by an attorney to modify as well as fro banking staff to modify terms to increase a mortgage and or change lending product to a balloon in an concerted effort to make homeowner foreclose which is also illegal, computer fraud and not the governments intended purpose or definition of a modification.'
DEVICE = 'cpu'

In [116]:
tokens = tokenizer_XLNET.tokenize(max_sen_in_train)

In [117]:
tokens = ' '.join([i.replace( '▁', '') for i in tokens])

In [ ]:
import spacy
spacy.prefer_gpu()
# spacy.prefer_cpu()
nlp = spacy.load("en_core_web_lg")
# 使用BERT进行fine-turning,文本也应该使用bert进行分词,以确保embedding的一致性.
# 这要求ner模块能够接受分好词的list,而不是自己进行分词,否则实体span获取不准确.

In [ ]:
def find_NER_in_spacy(raw_content, nlp = nlp, tokensize=True, ner=False, exclude_list = ['PERCENT', 'MONEY', 'QUANTITY', 'ORDINAL', 'CARDINAL']):
    '''使用spacy进行NER.
    标注解释: https://spacy.io/api/annotation
    '''
    res_nlp = nlp(raw_content)
    tokens = []
    if tokensize:
        tokens =  [str(i) for i in res_nlp.doc]
    
    entities_list = []
    if ner:
        for item in res_nlp.ents:
            if item.label_ in exclude_list: continue
            entities_dict = {}
            entities_dict['type'] = item.label_
            entities_dict['span_start'] = item.start
            entities_dict['content'] = item.text
            entities_dict['span_end'] = item.end
            entities_list.append(entities_dict)
        # print(dir(item))
    return tokens, entities_list

In [ ]:
ts, es = find_NER_in_spacy(max_sen_in_train, ner=True)
print(ts)
es

# XLNET测试

In [8]:
from transformers import AutoConfig, AutoModel, BertConfig, AutoTokenizer
from transformers import XLNetForQuestionAnswering

In [2]:
model_name = 'xlnet-base-cased'
config = AutoConfig.from_pretrained(model_name)

In [10]:
# tokenizer_XLNET = AutoTokenizer.from_pretrained(model_name)
model_XLNET = XLNetForQuestionAnswering.from_pretrained(model_name)

In [141]:
test = tokenizer_XLNET.encode('hello i am jim','who are you',add_special_tokens=True)
test = tokenizer_XLNET.decode(test)
test

'hello i am jim<sep> who are you<sep><cls>'

In [12]:
model_XLNET.state_dict().keys()

odict_keys(['transformer.mask_emb', 'transformer.word_embedding.weight', 'transformer.layer.0.rel_attn.q', 'transformer.layer.0.rel_attn.k', 'transformer.layer.0.rel_attn.v', 'transformer.layer.0.rel_attn.o', 'transformer.layer.0.rel_attn.r', 'transformer.layer.0.rel_attn.r_r_bias', 'transformer.layer.0.rel_attn.r_s_bias', 'transformer.layer.0.rel_attn.r_w_bias', 'transformer.layer.0.rel_attn.seg_embed', 'transformer.layer.0.rel_attn.layer_norm.weight', 'transformer.layer.0.rel_attn.layer_norm.bias', 'transformer.layer.0.ff.layer_norm.weight', 'transformer.layer.0.ff.layer_norm.bias', 'transformer.layer.0.ff.layer_1.weight', 'transformer.layer.0.ff.layer_1.bias', 'transformer.layer.0.ff.layer_2.weight', 'transformer.layer.0.ff.layer_2.bias', 'transformer.layer.1.rel_attn.q', 'transformer.layer.1.rel_attn.k', 'transformer.layer.1.rel_attn.v', 'transformer.layer.1.rel_attn.o', 'transformer.layer.1.rel_attn.r', 'transformer.layer.1.rel_attn.r_r_bias', 'transformer.layer.1.rel_attn.r_s_bia

In [134]:
model_input = tokenizer_XLNET.encode_plus(max_sen_in_train,
                                          add_special_tokens=True,
                                         return_tensors='pt')

In [135]:
outputs = model_XLNET(**model_input)

In [136]:
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
last_hidden_states.shape

torch.Size([1, 687, 768])

In [173]:
def get_features_from_XLNET(text,text_pair=None,
                            tokenizer = tokenizer_XLNET,
                            model = model_XLNET,
                            add_special_tokens = True,
                           device = 'cuda'):
    
    model_input = tokenizer_XLNET.encode_plus(text,text_pair,
                                        add_special_tokens=add_special_tokens,
                                        return_tensors='pt')
    print(tokenizer_XLNET.encode(text,text_pair ,add_special_tokens=add_special_tokens))
    
    model_input = {k:v.to(device) for k,v in model_input.items()}
    model.to(device)
    
    last_hidden_state = model(**model_input)[0]
    
    return last_hidden_state

In [174]:
last_hidden_state = get_features_from_XLNET('hello i am jim. who')

[24717, 17, 150, 569, 17, 2030, 98, 9, 61, 4, 3]


In [175]:
last_hidden_state.shape

torch.Size([1, 11, 768])

In [176]:
[tokenizer_XLNET.decode(i) for i in [24717, 17, 150, 569, 17, 2030, 98, 9, 61, 4, 3]]

['hello', '', 'i', 'am', '', 'ji', 'm', '.', 'who', '<sep>', '<cls>']

In [177]:
last_hidden_state[:,-1,:].shape

torch.Size([1, 768])

In [178]:
last_hidden_state2 = get_features_from_XLNET('hello i am jim. who',add_special_tokens=False)

[24717, 17, 150, 569, 17, 2030, 98, 9, 61]


In [179]:
last_hidden_state2.shape

torch.Size([1, 9, 768])

In [187]:
for i,k in model_XLNET.state_dict().items():
    print(i, k.shape)

mask_emb torch.Size([1, 1, 768])
word_embedding.weight torch.Size([32000, 768])
layer.0.rel_attn.q torch.Size([768, 12, 64])
layer.0.rel_attn.k torch.Size([768, 12, 64])
layer.0.rel_attn.v torch.Size([768, 12, 64])
layer.0.rel_attn.o torch.Size([768, 12, 64])
layer.0.rel_attn.r torch.Size([768, 12, 64])
layer.0.rel_attn.r_r_bias torch.Size([12, 64])
layer.0.rel_attn.r_s_bias torch.Size([12, 64])
layer.0.rel_attn.r_w_bias torch.Size([12, 64])
layer.0.rel_attn.seg_embed torch.Size([2, 12, 64])
layer.0.rel_attn.layer_norm.weight torch.Size([768])
layer.0.rel_attn.layer_norm.bias torch.Size([768])
layer.0.ff.layer_norm.weight torch.Size([768])
layer.0.ff.layer_norm.bias torch.Size([768])
layer.0.ff.layer_1.weight torch.Size([3072, 768])
layer.0.ff.layer_1.bias torch.Size([3072])
layer.0.ff.layer_2.weight torch.Size([768, 3072])
layer.0.ff.layer_2.bias torch.Size([768])
layer.1.rel_attn.q torch.Size([768, 12, 64])
layer.1.rel_attn.k torch.Size([768, 12, 64])
layer.1.rel_attn.v torch.Size([7

In [1]:
len(model_XLNET.state_dict())

NameError: name 'model_XLNET' is not defined

# 试验: BertForQuestionAnswering

In [5]:
from transformers import BertForQuestionAnswering, BertTokenizer

In [6]:
model_QA = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer_QA = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
model_input = tokenizer_QA.encode_plus(question,
                                      text,
                                      return_tensors='pt')
start_scores, end_scores = model_QA(**model_input)

In [8]:
start_scores

tensor([[ 0.2292,  0.1850,  0.5994,  0.1678,  0.4288,  0.2153,  0.4109,  0.0382,
          0.4649,  0.4879,  0.2751,  0.0596, -0.0334,  0.5149]],
       grad_fn=<SqueezeBackward1>)

In [10]:
end_scores

tensor([[-0.2515,  0.3538,  0.1440,  0.1174,  0.3960, -0.0503, -0.5198,  0.0995,
          0.2090,  0.1094, -0.1096, -0.1006,  0.0448, -0.3144]],
       grad_fn=<SqueezeBackward1>)

## 配置小型模型

In [26]:
from transformers import BertConfig, BertForQuestionAnswering

In [30]:
my_config = BertConfig()

In [31]:
my_config.max_length = 1024 # get ans span
my_config.max_position_embeddings = 1024
my_config.num_hidden_layers = 2
my_config.num_attention_heads = 6

In [32]:
my_model = BertForQuestionAnswering(my_config)

In [ ]:
ques_feat = torch.randn([3,5,11])

In [34]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_ids = tokenizer.encode(question, text)
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])


In [45]:
test_x = torch.randn([3,5,768])
test_attn_mask = [
    [1,1,1,0,0],
    [1,1,0,0,0],
    [1,1,1,1,0]
]
test_span = [
    [1,4],
    [0,3],
    [2,4]
]

model_input = {}
model_input['start_positions'], model_input['end_positions'] = torch.tensor(test_span).split(1, dim = -1)
model_input['attention_mask'] = torch.tensor(test_attn_mask)
model_input['inputs_embeds'] = test_x

In [46]:
my_model(**model_input)

(tensor(1.7547, grad_fn=<DivBackward0>),
 tensor([[ 0.2590, -0.1622, -0.6363,  0.5475, -0.0411],
         [ 0.3774,  0.3416, -0.1097,  0.5342, -0.4611],
         [-0.6236,  0.8827,  0.1005, -0.8110,  0.8700]],
        grad_fn=<SqueezeBackward1>),
 tensor([[ 0.3241, -0.0710,  0.6727,  0.3628, -0.2442],
         [ 0.2463,  0.7926,  0.0895,  0.7744,  0.2147],
         [-0.4006, -0.4710,  0.5912, -0.0770, -0.4697]],
        grad_fn=<SqueezeBackward1>))

# 测试Dataset与字符串

结果:

```python
TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class '__main__.test_ABC'>
```

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

import numpy as np

In [4]:
class test_ABC(object):
    def __init__(self):
        self.data = np.random.randn(1,3)        

In [17]:
strr = [
    ['hello', 'i', 'am'],
    ['old1', 'dog'],
    ['old2', 'dog'],
    ['old3', 'dog'],
    ['old4', 'dog'],
    ['old5', 'dog'],
    ['old6', 'dog'],
    ['old7', 'dog'],
    ['old8', 'dog'],
    ['old9', 'dog'],
]

class Test_Dateset(Dataset):
    def __init__(self, max_num=5000, max_seq=512, vocab_size=30522, label_num=2):
        self.max_num = max_num
        self.max_seq = max_seq
        self.vocab_size = vocab_size
        self.label_num = label_num
        
        self.trains = [test_ABC() for i in range(10)]
        
    def __len__(self):
        return len(self.trains)
    
    def __getitem__(self, index):
        data = torch.randint(0,self.vocab_size,(self.max_seq,), device='cuda').unsqueeze(0)
        labels = torch.randint(0,self.label_num-1,(1,), device='cuda')
        return {'input_ids':data,
                'labels':labels,
               'test': strr[index]}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

# def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cuda'): 
#     dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
#                             shuffle=shuffle, drop_last=drop_last)

#     for data_dict in dataloader:
#         out_data_dict = {}
#         for name, _tensor in data_dict.items():
#             out_data_dict[name] = data_dict[name].squeeze(1).to(device)
#         yield out_data_dict


In [18]:
dataset = Test_Dateset()
batch_size = 3

for item in DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=True, drop_last=True):
    print(item)


{'input_ids': tensor([[[28500,  7537, 28433,  ..., 25806, 12358,  6663]],

        [[ 4321,  1716, 26861,  ..., 29733, 13880,    31]],

        [[25880,  1496,  7014,  ..., 20770, 16101, 30424]]], device='cuda:0'), 'labels': tensor([[0],
        [0],
        [0]], device='cuda:0'), 'test': [('old5', 'old3', 'old1'), ('dog', 'dog', 'dog')]}
{'input_ids': tensor([[[19712,  7453, 29928,  ...,  3488, 13098,   999]],

        [[ 1767, 27465, 12974,  ...,   353, 24580,  8575]],

        [[20590, 10373, 21864,  ..., 21217, 14208, 10837]]], device='cuda:0'), 'labels': tensor([[0],
        [0],
        [0]], device='cuda:0'), 'test': [('old4', 'old8', 'hello'), ('dog', 'dog', 'i')]}
{'input_ids': tensor([[[ 7880, 13294, 13267,  ..., 20274,  7448, 22592]],

        [[12936, 10716,  7366,  ..., 20027, 17670, 20574]],

        [[16402, 29196, 19612,  ..., 20883, 25794, 19608]]], device='cuda:0'), 'labels': tensor([[0],
        [0],
        [0]], device='cuda:0'), 'test': [('old7', 'old2', 'old9'),

# nomalizations比较

In [11]:
import torch
from torch import nn

x = torch.rand(100, 16, 784)

# Batch Normalization层,因为输入是将高度H和宽度W合成了一个维度,所以这里用1d
layer = nn.BatchNorm1d(794)  # 传入通道数
out = layer(x)

# 全局的均值mu
print(layer.running_mean)
# 全局的方差sigma^2
print(layer.running_var)

RuntimeError: running_mean should contain 16 elements not 794

In [3]:
out.shape

torch.Size([100, 16, 784])

In [26]:
input = torch.randn(5, 200, 32)
# With Learnable Parameters
m1 = nn.LayerNorm(input.size()[1:])
# Without Learnable Parameters
m2 = nn.LayerNorm(input.size()[1:], elementwise_affine=False)
# Normalize over last two dimensions
m3 = nn.LayerNorm([200, 32])
# Normalize over last dimension of size 10
m4 = nn.LayerNorm([200])

In [27]:
output = m1(input)
output.shape

torch.Size([5, 200, 32])

In [28]:
output = m2(input)
output.shape

torch.Size([5, 200, 32])

In [29]:
output = m3(input)
output.shape

torch.Size([5, 200, 32])

In [30]:
output = m4(input)
output.shape

RuntimeError: Given normalized_shape=[200], expected input with shape [*, 200], but got input of size[5, 200, 32]

In [17]:
from transformers import AutoTokenizer, AutoModel
model_path = 'data/models/bert-base-cased'

In [18]:
tokenizer_bert = AutoTokenizer.from_pretrained(model_path)
model_bert = AutoModel.from_pretrained(model_path)

In [19]:
tokenizer_bert.batch_encode_plus([("hello","i am jim."),
                                  ("how", "are you")],
                                return_tensors='pt',
                                max_length = 20,
                                pad_to_max_length = True,
                                return_special_tokens_masks = True)

{'special_tokens_mask': tensor([[1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101, 19082,   102,   178,  1821, 23220,  1306,   119,   102,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
         [  101,  1293,   102,  1132,  1128,   102,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [26]:
tokenizer_bert.encode('china as good', add_special_tokens=True)

[101, 5144, 1161, 1112, 1363, 102]

In [28]:
tokenizer_bert.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [29]:
tokenizer_bert.encode('[UNK] [SEP] [PAD] [CLS] [MASK]')

[101, 100, 102, 0, 101, 103, 102]

In [33]:
tokenizer_bert.decode([i for i in range(0, 1000)])

'[PAD] [unused1] [unused2] [unused3] [unused4] [unused5] [unused6] [unused7] [unused8] [unused9] [unused10] [unused11] [unused12] [unused13] [unused14] [unused15] [unused16] [unused17] [unused18] [unused19] [unused20] [unused21] [unused22] [unused23] [unused24] [unused25] [unused26] [unused27] [unused28] [unused29] [unused30] [unused31] [unused32] [unused33] [unused34] [unused35] [unused36] [unused37] [unused38] [unused39] [unused40] [unused41] [unused42] [unused43] [unused44] [unused45] [unused46] [unused47] [unused48] [unused49] [unused50] [unused51] [unused52] [unused53] [unused54] [unused55] [unused56] [unused57] [unused58] [unused59] [unused60] [unused61] [unused62] [unused63] [unused64] [unused65] [unused66] [unused67] [unused68] [unused69] [unused70] [unused71] [unused72] [unused73] [unused74] [unused75] [unused76] [unused77] [unused78] [unused79] [unused80] [unused81] [unused82] [unused83] [unused84] [unused85] [unused86] [unused87] [unused88] [unused89] [unused90] [unused91] [

In [35]:
tokenizer_bert.decode([127])

'6'

In [36]:
'sssSS'.lower()

'sssss'

# Albert

In [9]:
from transformers import AlbertForQuestionAnswering, AlbertTokenizer
model_path = 'data/models/albert-base-v2'

In [10]:
tokenizer_albert = AlbertTokenizer.from_pretrained(model_path)
model_albert = AlbertForQuestionAnswering.from_pretrained(model_path)

In [11]:
text = [('hello, i am 88899', 'who are you 88899?'),]
text_str = 'hello, i am 88899, who are you 88899?'

In [12]:
tokenizer_albert.tokenize(text_str)

['▁hello',
 ',',
 '▁i',
 '▁am',
 '▁8',
 '88',
 '99',
 ',',
 '▁who',
 '▁are',
 '▁you',
 '▁8',
 '88',
 '99',
 '?']

In [13]:
tokenizer_albert.encode('China')

[2, 998, 3]

In [15]:
tokenizer_albert.decode([i for i in range(50)])

'<pad><unk>[CLS][SEP][MASK]()"-.–£€  the, of ands in to a\' was he is for on as with that i it his by at her fromt she an had youd be: were but thisi'

In [21]:
tokenizer_albert.batch_encode_plus(text, return_tensors='pt')

{'input_ids': tensor([[    2, 10975,    15,    31,   589,   469,  3020,  3483,     3,    72,
             50,    42,   469,  3020,  3483,    60,     3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [27]:
model_albert.state_dict

<bound method Module.state_dict of AlbertForQuestionAnswering(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_f

In [1]:
from QA_models import AutoQuestionAnswering

In [38]:
model = AutoQuestionAnswering.from_pretrained('data/models/bert-large-uncased-whole-word-masking')
for k,v in model.state_dict().items():
    print(k, v.shape)
    

transformer.embeddings.word_embeddings.weight torch.Size([30522, 1024])
transformer.embeddings.position_embeddings.weight torch.Size([512, 1024])
transformer.embeddings.token_type_embeddings.weight torch.Size([2, 1024])
transformer.embeddings.LayerNorm.weight torch.Size([1024])
transformer.embeddings.LayerNorm.bias torch.Size([1024])
transformer.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.self.query.bias torch.Size([1024])
transformer.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.self.key.bias torch.Size([1024])
transformer.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.self.value.bias torch.Size([1024])
transformer.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.output.dense.bias torch.Size([1024])
transformer.encoder.layer.0.attention.output.Lay

In [39]:
model = AutoQuestionAnswering.from_pretrained('data/models/bert-large-cased-whole-word-masking')
for k,v in model.state_dict().items():
    print(k, v.shape)
    

transformer.embeddings.word_embeddings.weight torch.Size([28996, 1024])
transformer.embeddings.position_embeddings.weight torch.Size([512, 1024])
transformer.embeddings.token_type_embeddings.weight torch.Size([2, 1024])
transformer.embeddings.LayerNorm.weight torch.Size([1024])
transformer.embeddings.LayerNorm.bias torch.Size([1024])
transformer.encoder.layer.0.attention.self.query.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.self.query.bias torch.Size([1024])
transformer.encoder.layer.0.attention.self.key.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.self.key.bias torch.Size([1024])
transformer.encoder.layer.0.attention.self.value.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.self.value.bias torch.Size([1024])
transformer.encoder.layer.0.attention.output.dense.weight torch.Size([1024, 1024])
transformer.encoder.layer.0.attention.output.dense.bias torch.Size([1024])
transformer.encoder.layer.0.attention.output.Lay

In [4]:
import torch


<function _VariableFunctions.zeros_like>

In [7]:
x = torch.tensor([1,2]).as_type()

AttributeError: 'Tensor' object has no attribute 'as_type'

# RoBerta

In [31]:
from transformers import RobertaForQuestionAnswering, RobertaTokenizer
model_path = 'data/models/roberta-base/'

In [33]:
tokenizer_Roberta = RobertaTokenizer.from_pretrained(model_path)
model_Roberta = RobertaForQuestionAnswering.from_pretrained(model_path)

In [34]:
text = [('hello, i am 88899', 'who are you 88899?'),]
text_str = 'hello, i am 88899, who are you 88899?'

In [35]:
tokenizer_Roberta.tokenize(text_str)

['hello',
 ',',
 'Ġi',
 'Ġam',
 'Ġ8',
 '88',
 '99',
 ',',
 'Ġwho',
 'Ġare',
 'Ġyou',
 'Ġ8',
 '88',
 '99',
 '?']

In [36]:
tokenizer_Roberta.batch_encode_plus(text, return_tensors='pt')

{'input_ids': tensor([[    0, 20760,     6,   939,   524,   290,  4652,  2831,     2,     2,
             54,    32,    47,   290,  4652,  2831,   116,     2]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [38]:
model_Roberta.state_dict

<bound method Module.state_dict of RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

In [16]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast
model_path = 'data/models/distilbert-base-uncased-distilled-squad'

In [17]:
tokenizer_Distil = DistilBertTokenizerFast.from_pretrained(model_path, local_files_only=True)
model_Distil = DistilBertForQuestionAnswering.from_pretrained(model_path, local_files_only=True)

In [47]:
text = [('hello, i am 88899', 'who are you 88899?'),]
text_str = 'hello, i am 88899, who are you 88899?'

In [48]:
tokenizer_Distil.tokenize(text_str)

['[CLS]',
 'hello',
 ',',
 'i',
 'am',
 '88',
 '##8',
 '##9',
 '##9',
 ',',
 'who',
 'are',
 'you',
 '88',
 '##8',
 '##9',
 '##9',
 '?',
 '[SEP]']

In [49]:
tokenizer_Distil.batch_encode_plus(text, return_tensors='pt', )

{'input_ids': tensor([[ 101, 7592, 1010, 1045, 2572, 6070, 2620, 2683, 2683,  102, 2040, 2024,
          2017, 6070, 2620, 2683, 2683, 1029,  102]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [50]:
model_Distil.state_dict

<bound method Module.state_dict of DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout